#### imports

In [1]:
import csv
import requests
import math
import pprint
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime, date, time
from IPython.display import clear_output

In [2]:
cwd = os.getcwd()
print cwd

/Users/jpy/Dropbox/_FLASK/__JUPYTER__/OCF


In [3]:
pp = pprint.PrettyPrinter(indent=2)

#### common functions

In [4]:
def findFromPathString(path_string, JSON, separator="/", debug=False):     
    paths_ = path_string.split(separator)
    data = JSON
    if debug :
        print "\n data : "
        print pp.pprint(data)
    try : 
        for i in range(0,len(paths_)):
            data = data[paths_[i]]
        return data
    except :
        return False

In [5]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

#### settings

In [6]:

root_url_corpos = "http://opencorporatefacts.fr/api/corporates"
# params_corpos = { 'page' : 1 , 'id' : None , 'CompanyNumber' : None }
results_path_corpos = "hydra:member"
output_filename_corpos = "datasets/corporations/corpos"
headers_corpos = {'accept': 'application/ld+json'}

root_url_accounts = "http://opencorporatefacts.fr/api/compte_de_resultats"
# params_accounts = { 'page' : 1, 'Corporate' : None }
results_path_accounts = "hydra:member"
output_filename_accounts = "datasets/account_results/accounts"
headers_accounts = {'accept': 'application/ld+json'}

root_url_companies = "http://api.cquest.org/company/" ### pass CompanyNumber after root_url
output_filename_companies = "datasets/companies_data/companies"


#### tests

#### main requests functions

In [11]:
def generate_filename( output_filename, page_treshold, split_files_treshold_pages, max_page, only_filename=False ) :
    
    start_page = page_treshold
    end_page = page_treshold + split_files_treshold_pages - 1
    if end_page > max_page : 
        end_page = max_page
    
    if only_filename : 
        prefix = ''
    else : 
        prefix = cwd + '/'
        
    filename = prefix + output_filename + '-pageStart_' + str(start_page) + '_to_' + str(end_page)
    
    return filename
    
def get_req_results ( 
    url, 
    pages_range, 
    number_of_pages, 
    results_path, 
    output_filename, 
    start_page=1, 
    split_files_treshold_pages=1000,
    debug=False, 
    out_formats=["csv"] 
    ) : 
    ''' 
    retrieve results from requests 
    and write csv 
    works with API platform only for now
    '''
    
    print "url : {}\n".format(url)
    
    start_time = datetime.now()
    
    ### count pages
    count = 0
    page_treshold = 0
    
    ### requests and writing file
    
    for page in pages_range : 
    
        if page > 0 and page >= start_page  : 
            
            ### treshold and split into several files
            if page % split_files_treshold_pages == 0 :
                page_treshold = page
                count = 0
                
            ### output files
            if "csv" in out_formats :
                file_data_csv_filename  = generate_filename( output_filename, page_treshold, split_files_treshold_pages, number_of_pages )
                file_data_csv  = file_data_csv_filename + '.csv'

            if "json" in out_formats : 
                file_data_json_filename = generate_filename( output_filename, page_treshold, split_files_treshold_pages, number_of_pages )
                file_data_json = file_data_json_filename + '.json'
            
            if debug == False : 
                clear_output( wait=True ) 
            
            resp = requests.get( url, params = { 'page' : page } ).json()
            results = findFromPathString( results_path, resp, debug=debug )
            
            page_time = datetime.now()
            delta_time = page_time - start_time
            
            filename = generate_filename( output_filename, page_treshold, split_files_treshold_pages, number_of_pages, only_filename=True )
            
            print "\nPage number : {} / {} \
                    \ndelta_time (after request) : {}\
                    \npage_treshold : {}\
                    \nfilename : {}\
                    \n".format(page, number_of_pages, delta_time, page_treshold, filename)
            
            if results : 
                # loop results
                for data in results : 

                    if debug : 
                        # print pp.pprint(data) 
                        print data
                        print
                    data_flat = flatten_json(data)
                    # print data_flat

                    ### CSV
                    if "csv" in out_formats : 

                        ### creates headers
                        if count == 0:
                            with open(file_data_csv, 'w') as csv_file :
                                csv_writer = csv.writer(csv_file, delimiter=';')
                                header = data.keys()
                                csv_writer.writerow(header)
                                count += 1
                        else : 
                            with open(file_data_csv, 'a') as csv_file :
                                csv_writer = csv.writer(csv_file, delimiter=';')
                                csv_writer.writerow( data.values() )
                                count += 1


                    ### JSON
                    if "json" in out_formats : 
                        if count == 0 :
                            with open(file_data_json, 'w') as json_file :
                                json_file.write(json.dumps( [data] ).encode())         #If empty, write an array
                                count += 1
                        else : 
                            with open(file_data_json, 'ab+') as json_file :
                                json_file.seek(-1,2)           
                                json_file.truncate()                           #Remove the last character, open the array
                                json_file.write(' , '.encode())                #Write the separator
                                json_file.write(json.dumps(data).encode())     #Dump the dictionary
                                json_file.write(']'.encode())                  #Close the array
                                count += 1


#### getting data / tests

In [12]:
### TEST / get accounts 
number_of_pages_accounts_test = 30
pages_range_test = range( number_of_pages_accounts_test )
get_req_results( 
    root_url_accounts, 
    pages_range_test, 
    number_of_pages_accounts_test,
    results_path_accounts, 
    output_filename_accounts+'_test', 
    start_page=1, 
    split_files_treshold_pages=10,
    debug=True, 
    out_formats=["json"] 
) 

url : http://opencorporatefacts.fr/api/compte_de_resultats


 data : 
{ u'@context': u'/api/contexts/Error',
  u'@type': u'hydra:Error',
  u'hydra:description': u'Warning: file_put_contents(/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php.5de98db1c5fa08.63637851): failed to open stream: No space left on device',
  u'hydra:title': u'An error occurred',
  u'trace': [ { u'args': [],
                u'class': u'',
                u'file': u'/var/www/OpenCorporateFacts/vendor/doctrine/common/lib/Doctrine/Common/Proxy/ProxyGenerator.php',
                u'function': u'',
                u'line': 291,
                u'namespace': u'',
                u'short_class': u'',
                u'type': u''},
              { u'args': [ [ u'object',
                             u'Doctrine\\ORM\\Mapping\\ClassMetadata'],
                           [ u'string',
                             u'/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG_

                u'class': u'ApiPlatform\\Core\\Hydra\\Serializer\\CollectionNormalizer',
                u'file': u'/var/www/OpenCorporateFacts/vendor/api-platform/core/src/Hydra/Serializer/PartialCollectionViewNormalizer.php',
                u'function': u'normalize',
                u'line': 47,
                u'namespace': u'ApiPlatform\\Core\\Hydra\\Serializer',
                u'short_class': u'CollectionNormalizer',
                u'type': u'->'},
              { u'args': [ [ u'object',
                             u'ApiPlatform\\Core\\Bridge\\Doctrine\\Orm\\Paginator'],
                           [u'string', u'jsonld'],
                           [ u'array',
                             { u'collection_operation_name': [ u'string',
                                                               u'get'],
                               u'input': [u'null', None],
                               u'operation_type': [ u'string',
                                                    u'co

              { u'args': [ [ u'array',
                             { u'achat_matiere_premiere_autre_appro_11': [ u'string',
                                                                           u'5809'],
                               u'achats_de_marchandises_10': [ u'string',
                                                               u'338287'],
                               u'autre_interet_et_produit_assimile_25': [ u'string',
                                                                          u'38'],
                               u'autres_achat_et_charges_externes_12': [ u'string',
                                                                         u'192117'],
                               u'autres_charges_20': [u'string', u'40'],
                               u'autres_produits_8': [u'string', u'23'],
                               u'benefice_44': [u'string', u'16224'],
                               u'charges_exceptionnelle_operation_capital_39': [ u'string


 data : 
{ u'@context': u'/api/contexts/Error',
  u'@type': u'hydra:Error',
  u'hydra:description': u'Warning: file_put_contents(/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php.5de98db2624410.19213080): failed to open stream: No space left on device',
  u'hydra:title': u'An error occurred',
  u'trace': [ { u'args': [],
                u'class': u'',
                u'file': u'/var/www/OpenCorporateFacts/vendor/doctrine/common/lib/Doctrine/Common/Proxy/ProxyGenerator.php',
                u'function': u'',
                u'line': 291,
                u'namespace': u'',
                u'short_class': u'',
                u'type': u''},
              { u'args': [ [ u'object',
                             u'Doctrine\\ORM\\Mapping\\ClassMetadata'],
                           [ u'string',
                             u'/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php']],
                u'class': u'Doct


 data : 
{ u'@context': u'/api/contexts/Error',
  u'@type': u'hydra:Error',
  u'hydra:description': u'Warning: file_put_contents(/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php.5de98db2a1bb07.85361878): failed to open stream: No space left on device',
  u'hydra:title': u'An error occurred',
  u'trace': [ { u'args': [],
                u'class': u'',
                u'file': u'/var/www/OpenCorporateFacts/vendor/doctrine/common/lib/Doctrine/Common/Proxy/ProxyGenerator.php',
                u'function': u'',
                u'line': 291,
                u'namespace': u'',
                u'short_class': u'',
                u'type': u''},
              { u'args': [ [ u'object',
                             u'Doctrine\\ORM\\Mapping\\ClassMetadata'],
                           [ u'string',
                             u'/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php']],
                u'class': u'Doct

...

                               u'DifferencesPositivesChange': [ u'string',
                                                                u'0'],
                               u'Dividende': [u'string', u'-1'],
                               u'DotationAmortissementImmobilisations': [ u'string',
                                                                          u'21615'],
                               u'DotationDepreciationActifCirculant': [ u'string',
                                                                        u'0'],
                               u'DotationDepreciationImmobilisations': [ u'string',
                                                                         u'0'],
                               u'DotationExceptionnelleAmortissementDepreciationProvision': [ u'string',
                                                                                              u'0'],
                               u'DotationProvisions': [u'string', u'0'],
              


 data : 
{ u'@context': u'/api/contexts/Error',
  u'@type': u'hydra:Error',
  u'hydra:description': u'Warning: file_put_contents(/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php.5de98dbbaf7129.33126055): failed to open stream: No space left on device',
  u'hydra:title': u'An error occurred',
  u'trace': [ { u'args': [],
                u'class': u'',
                u'file': u'/var/www/OpenCorporateFacts/vendor/doctrine/common/lib/Doctrine/Common/Proxy/ProxyGenerator.php',
                u'function': u'',
                u'line': 291,
                u'namespace': u'',
                u'short_class': u'',
                u'type': u''},
              { u'args': [ [ u'object',
                             u'Doctrine\\ORM\\Mapping\\ClassMetadata'],
                           [ u'string',
                             u'/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php']],
                u'class': u'Doct


 data : 
{ u'@context': u'/api/contexts/Error',
  u'@type': u'hydra:Error',
  u'hydra:description': u'Warning: file_put_contents(/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php.5de98dbc0706e1.33901430): failed to open stream: No space left on device',
  u'hydra:title': u'An error occurred',
  u'trace': [ { u'args': [],
                u'class': u'',
                u'file': u'/var/www/OpenCorporateFacts/vendor/doctrine/common/lib/Doctrine/Common/Proxy/ProxyGenerator.php',
                u'function': u'',
                u'line': 291,
                u'namespace': u'',
                u'short_class': u'',
                u'type': u''},
              { u'args': [ [ u'object',
                             u'Doctrine\\ORM\\Mapping\\ClassMetadata'],
                           [ u'string',
                             u'/var/www/OpenCorporateFacts/var/cache/dev/doctrine/orm/Proxies/__CG__AppEntityCorporate.php']],
                u'class': u'Doct

                             u'Symfony\\Component\\HttpFoundation\\Request'],
                           [u'integer', 1]],
                u'class': u'Symfony\\Component\\HttpKernel\\HttpKernel',
                u'file': u'/var/www/OpenCorporateFacts/vendor/symfony/http-kernel/HttpKernel.php',
                u'function': u'handleRaw',
                u'line': 66,
                u'namespace': u'Symfony\\Component\\HttpKernel',
                u'short_class': u'HttpKernel',
                u'type': u'->'},
              { u'args': [ [ u'object',
                             u'Symfony\\Component\\HttpFoundation\\Request'],
                           [u'integer', 1],
                           [u'boolean', True]],
                u'class': u'Symfony\\Component\\HttpKernel\\HttpKernel',
                u'file': u'/var/www/OpenCorporateFacts/vendor/symfony/http-kernel/Kernel.php',
                u'function': u'handle',
                u'line': 188,
                u'namespace': u'Symfony\

                               u'input': [u'null', None],
                               u'operation_type': [ u'string',
                                                    u'collection'],
                               u'output': [u'null', None],
                               u'request_uri': [ u'string',
                                                 u'/api/compte_de_resultats?page=29'],
                               u'resource_class': [ u'string',
                                                    u'App\\Entity\\CompteDeResultat'],
                               u'resources': [ u'object',
                                               u'ApiPlatform\\Core\\Serializer\\ResourceList'],
                               u'resources_to_push': [ u'object',
                                                       u'ApiPlatform\\Core\\Serializer\\ResourceList'],
                               u'uri': [ u'string',
                                         u'http://opencorporatefacts.fr/api/com

In [9]:
### TEST / get corpos 
number_of_pages_corpos_test = 30
pages_range_test = range(number_of_pages_corpos_test)
get_req_results( 
    root_url_corpos, 
    pages_range_test, 
    number_of_pages_corpos_test,
    results_path_corpos, 
    output_filename_corpos+'_test', 
    start_page=1, 
    split_files_treshold_pages=10,
    debug=True, 
    out_formats=["json"] 
) 

url : http://opencorporatefacts.fr/api/corporates


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/2',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'414119735',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/1'],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'',
                       u'Name': u'AYMING',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414119735',
                       u'documentDeReferences': [],
                       u'id': 2,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/3',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'419373055',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultat


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/32',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'347531402',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'4730Z',
                       u'Name': u'AMALLY',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/347531402',
                       u'documentDeReferences': [],
                       u'id': 32,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/33',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'490848223',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'4


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/62',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'518552435',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/22'],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'7112B',
                       u'Name': u'G ENERGY',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/518552435',
                       u'documentDeReferences': [],
                       u'id': 62,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/63',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'490902632',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
          


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/92',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'794462770',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/5636',
                                                u'/api/compte_de_resultats/214521',
                                                u'/api/compte_de_resultats/214520'],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'6420Z',
                       u'Name': u'HOLDING MJJ',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/794462770',
                       u'documentDeReferences': [],
                       u'id': 92,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/93',
                       u'@type': u'Corpora


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/122',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'819816166',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'5610C',
                       u'Name': u'1672 House Coffee',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/819816166',
                       u'documentDeReferences': [],
                       u'id': 122,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/123',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'808306245',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'Indu

...


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/722',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'341134757',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'4642Z',
                       u'Name': u'PATAGONIA EUROPE',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/341134757',
                       u'documentDeReferences': [],
                       u'id': 722,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/723',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'802968040',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/216993'],
                       u'CorporateShareholders':


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/752',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'810592865',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'7112B',
                       u'Name': u'INGENIERIE ET CONSEIL EN ETUDE TECHNIQUE',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/810592865',
                       u'documentDeReferences': [],
                       u'id': 752,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/753',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'577380405',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/5698'],
                       u'C


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/782',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'494629298',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/229'],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'6820B',
                       u'Name': u'IMMOBILIERE 27 AC',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/494629298',
                       u'documentDeReferences': [],
                       u'id': 782,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/783',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'478564198',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/812',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'414633164',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/104408',
                                                u'/api/compte_de_resultats/238'],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'6420Z',
                       u'Name': u'SOCIETE FINANCIERE RENAUD',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414633164',
                       u'documentDeReferences': [],
                       u'id': 812,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/813',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'812102432',
        


 data : 
{ u'@context': u'/api/contexts/Corporate',
  u'@id': u'/api/corporates',
  u'@type': u'hydra:Collection',
  u'hydra:member': [ { u'@id': u'/api/corporates/842',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'500998778',
                       u'ComptesDeResultats': [],
                       u'CorporateShareholders': [],
                       u'IndustryCode': u'9511Z',
                       u'Name': u'DARE-DARE INFORMATIQUE',
                       u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/500998778',
                       u'documentDeReferences': [],
                       u'id': 842,
                       u'shareholdings': []},
                     { u'@id': u'/api/corporates/843',
                       u'@type': u'Corporate',
                       u'CompanyNumber': u'378674477',
                       u'ComptesDeResultats': [ u'/api/compte_de_resultats/5706'],
                       u'CorporateShareholde

#### getting data / real

In [ ]:
### get corpos data

number_of_results_corpos = 577297
results_per_page = 30
number_of_pages_corpos = int( math.ceil( number_of_results_corpos / results_per_page ) )
print number_of_pages_corpos
pages_to_scrap_corpos = range( number_of_pages_corpos + 1)

start_page_corpos = 1
file_suffix_corpos = '_07'
file_treshold_corpos = 2500

get_req_results(  
    root_url_corpos, 
    pages_to_scrap_corpos, 
    number_of_pages_corpos,
    results_path_corpos, 
    output_filename_corpos+file_suffix_corpos, 
    start_page=start_page_corpos, 
    split_files_treshold_pages=file_treshold_corpos,
    debug=False, 
    out_formats=["json"] 
) 


Page number : 19191 / 19243                     
delta_time (after request) : 2:52:50.467055                    
page_treshold : 17500                    
filename : datasets/corporations/corpos_07-pageStart_17500_to_19243                    



In [ ]:
### get accounts data

number_of_results_account = 316182
results_per_page = 30
number_of_pages_accounts = int( math.ceil( number_of_results_account / results_per_page ) )
print number_of_pages_accounts
pages_to_scrap_accounts = range( number_of_pages_accounts + 1 )

start_page_accounts = 1
file_suffix_accounts = '_scrap06'
file_treshold_accounts = 500

get_req_results( 
    root_url_accounts, 
    pages_to_scrap_accounts, 
    number_of_pages_accounts,
    results_path_accounts, 
    output_filename_accounts+file_suffix_accounts, 
    start_page=start_page_accounts, 
    split_files_treshold_pages=file_treshold_accounts,
    debug=False, 
    out_formats=["json"] 
) 